In [ ]:
import os
import sys

src_path = os.path.abspath("../..")
print(src_path)
sys.path.append(src_path)

In [ ]:
from src.utils import create_directory, raw_data_path, processed_data_path, set_seed

In [ ]:
set_seed(seed=42)

In [ ]:
import pandas as pd

In [ ]:
mimic_iv_path = os.path.join(raw_data_path, "physionet.org/files/mimiciv/2.2")
output_path = os.path.join(processed_data_path, "mimic4")

In [ ]:
cohort = pd.read_csv(os.path.join(output_path, "cohort.csv"))
print(cohort.shape)
cohort.head()

In [ ]:
cohort["hadm_intime"] = pd.to_datetime(cohort["hadm_intime"])
cohort["hadm_outtime"] = pd.to_datetime(cohort["hadm_outtime"])
cohort["stay_intime"] = pd.to_datetime(cohort["stay_intime"])
cohort["stay_outtime"] = pd.to_datetime(cohort["stay_outtime"])

In [ ]:
hadm_ids = set(cohort.hadm_id.unique().tolist())
len(hadm_ids)

helper

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
from pandarallel import pandarallel

In [ ]:
pandarallel.initialize(progress_bar=True)

In [ ]:
def save_group(group_df, hadm_id, event_type):
    file_path = f"{output_path}/event_{event_type}/event_{int(hadm_id)}.csv"
    group_df.to_csv(file_path, index=False)
    return True

dict

In [ ]:
d_items = pd.read_csv(os.path.join(mimic_iv_path, "icu/d_items.csv.gz"))
print(d_items.shape)
d_items.head()

In [ ]:
d_labitems = pd.read_csv(os.path.join(mimic_iv_path, "hosp/d_labitems.csv.gz"))
print(d_labitems.shape)
d_labitems.head()

## transfers

In [ ]:
event_type = "transfers"

In [ ]:
!rm -r {output_path}/event_{event_type}

In [ ]:
create_directory(f"{output_path}/event_{event_type}")

In [ ]:
df = pd.read_csv(os.path.join(mimic_iv_path, f"hosp/{event_type}.csv.gz"))
print(df.shape)
df.head()

In [ ]:
df = df.merge(cohort[["hadm_id", "hadm_intime"]], on="hadm_id", how="inner")
print(df.shape)
df.head()

In [ ]:
df["intime"] = pd.to_datetime(df["intime"])
df["outtime"] = pd.to_datetime(df["outtime"])
df["timestamp"] = (df.intime - df.hadm_intime).dt.total_seconds() / 3600
df.head()

In [ ]:
df = df.sort_values(["subject_id", "hadm_id", "timestamp"], ascending=True)
df.head()

In [ ]:
df["hadm_id"] = df["hadm_id"].astype(int)
df["eventtype"] = df["eventtype"].replace("ED", "admit")
df.head()

In [ ]:
df.isna().sum()

In [ ]:
def generate_event_value(x):
    s = x.eventtype
    if not pd.isna(x.careunit):
        s += f" to {x.careunit}"
    return s

In [ ]:
meta_cols = ["eventtype", "careunit"]
for c in meta_cols:
    df["meta_" + c] = df[c]
meta_cols = ["meta_" + c for c in meta_cols]

In [ ]:
df["timestamp_avail"] = df["timestamp"]

In [ ]:
generate_event_value(df.iloc[5])

In [ ]:
df["event_type"] = event_type
df["event_value"] = df.parallel_apply(generate_event_value, axis=1)

In [ ]:
df[df.hadm_id == 29079034]

In [ ]:
df.groupby("hadm_id").timestamp.count().describe()

In [ ]:
groups = df.groupby("hadm_id")
    
with ThreadPoolExecutor(max_workers=4) as executor:
    for hadm_id, group_df in tqdm(groups, total=groups.ngroups):
        future = executor.submit(
            save_group, 
            group_df[["hadm_id", "event_type", "timestamp", "event_value", "timestamp_avail"] + meta_cols], 
            hadm_id, 
            event_type
        )

In [ ]:
!ls -1 {output_path}/event_{event_type} | wc -l